In [172]:
import numpy as np
import pandas as pd
data = pd.read_csv("TSLA_18_year_testing_signal.csv")
dt=data
data.drop(data.columns[[0]],axis=1,inplace=True)
window_size = int(len(data)/(8))
splittrain = []
splittest = [] 
for i in range(0, len(data)-5*window_size,window_size):
    splittrain.append(data[i:i+5*window_size])
    splittest.append(data[i+5*window_size:i+6*window_size])

In [173]:
from scipy.optimize import NonlinearConstraint, Bounds, LinearConstraint
import sympy as sp
w1=sp.symbols('w1')
w2=sp.symbols('w2')
w3=sp.symbols('w3')
w4=sp.symbols('w4')
w5=sp.symbols('w5')
w6=sp.symbols('w6')
w7=sp.symbols('w7')
w8=sp.symbols('w8')
w9=sp.symbols('w9')
w10=sp.symbols('w10')
td=sp.symbols('td')

In [174]:
def const(p):
    w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,td = p
    return w1+w2+w3+w4+w5+w6+w7+w8+w9+w10
nlc = NonlinearConstraint(const, 1 ,1)
import numpy as np
from scipy.optimize import differential_evolution
from scipy.optimize import NonlinearConstraint, Bounds

bounds = [[0, 1], [0, 1],[0, 1], [0, 1],[0, 1],[0, 1],[0, 1], [0, 1],[0, 1],[0,1],[0,0.3]]

In [182]:
coll=[]

def func(p):
    global S,B,resl,resfl
    S=0
    B=0
    resl=[]
    resfl=[]
    w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,td = p
    mon=100000
    amount=0
    for i in range(0,len(dfo)):
        r = ((w1*sl1[i]+w2*sl2[i]+w3*sl3[i]+w4*sl4[i]+w5*sl5[i]+w6*sl6[i]+w7*sl7[i]+w8*sl8[i]+w9*sl9[i]+w10*sl10[i])/(w1+w2+w3+w4+w5+w6+w7+w8+w9+w10))#w1=sp.symbols('w1')
        if i!=0:
            if dfo['adj_close_price'][i]<dfo['adj_close_price'][i-1]*0.8:
                mon+=amount*dfo['adj_close_price'][i]*0.998
                amount=0
        if r>td:
            sumall=mon//dfo['adj_close_price'][i]
            amount+=sumall
            mon-=sumall*dfo['adj_close_price'][i]*1.002
            B+=1
        elif r<-td:
            mon+=amount*dfo['adj_close_price'][i]*0.998
            amount=0
            S+=1
        else:
            pass
    ret=mon+amount*dfo['adj_close_price'][len(dfo)-1]
    return -((ret-100000)/1000)
def fobj():
    global sl1,sl2,sl3,sl4,sl5,sl6,sl7,sl8,sl9,sl10,dfo
    for i in range(len(splittrain)):
        dfo=splittrain[i]
        dfo=dfo.reset_index()
        sl1=dfo['sma89_signal']
        sl2=dfo['obv_macd_signal']
        sl3=dfo['cdc_signal']
        sl4=dfo['rsi_signal']
        sl5=dfo['renko_signal']
        sl6=dfo['sar_signal']
        sl7=dfo['macd_signal']
        sl8=dfo['stoch_signal']
        sl9=dfo['obv_renko_signal']
        sl10=dfo['aroon_signal']
        result = differential_evolution(func, bounds,constraints=(nlc),maxiter=1500,popsize=80,recombination=0.8) 
        coll.append([result.x,-result.fun,i])
        print([result.x,-result.fun,i])
        a=result.x
        b=i
        td=a[10]
        w=a[0:10]
        df=splittest[b]
        decision_d_l=[]
        il=[]
        global count1,count2,count3,sell_idx,buy_idx
        mon=100000
        amount=0
        sumall=0
        df=df.reset_index()
        sl1=df['sma89_signal']
        sl2=df['obv_macd_signal']
        sl3=df['cdc_signal']
        sl4=df['rsi_signal']
        sl5=df['renko_signal']
        sl6=df['sar_signal']
        sl7=df['macd_signal']
        sl8=df['stoch_signal']
        sl9=df['obv_renko_signal']
        sl10=df['aroon_signal']

        for i in range(0,len(df)):
            decision_d= ((w[0]*sl1[i]+w[1]*sl2[i]+w[2]*sl3[i]+w[3]*sl4[i]+w[4]*sl5[i]+w[5]*sl6[i]+w[6]*sl7[i]+w[7]*sl8[i]+w[8]*sl9[i]+w[9]*sl10[i])/sum(w))
            if i!=0:
                if df['adj_close_price'][i]<df['adj_close_price'][i-1]*0.8:
                        mon+=amount*df['Close'][i]*0.998
                        amount=0
                        sell_idx.append(i)
            if decision_d>td:
                sumall=mon//df['adj_close_price'][i]
                amount+=sumall
                mon-=sumall*df['adj_close_price'][i]*1.002
                count1+=1
                buy_idx.append(i)
            elif decision_d<-td:
                mon+=amount*df['adj_close_price'][i]*0.998
                amount=0
                count2+=1
                sell_idx.append(i)
            else:
                count3+=1
        decision_d_l.append(mon+amount*df['adj_close_price'][i])
        il.append(i)
        ret=mon+amount*df['adj_close_price'][len(df)-1]
        stocknum=100000/dfo['adj_close_price'][0]
        print("training",(stocknum*(dfo['adj_close_price'][len(dfo)-1])+(100000-stocknum*dfo['adj_close_price'][0]))-100000)
        stocknum=100000/df['adj_close_price'][0]
        print("test",(stocknum*(df['adj_close_price'][len(df)-1])+(100000-stocknum*df['adj_close_price'][0]))-100000)
        print ("TEST",b,(ret-100000)/1000)

        
# def testinwindow(a,b):
#     td=a[10]
#     w=a[0:10]
#     df=splittest[0]

#     decision_d_l=[]
#     il=[]
#     global count1,count2,count3,sell_idx,buy_idx,sl1
#     mon=100000
#     amount=0
#     sumall=0
#     df=df.reset_index()
#     sl1=df['sma89_signal']
#     sl2=df['obv_macd_signal']
#     sl3=df['cdc_signal']
#     sl4=df['rsi_signal']
#     sl5=df['renko_signal']
#     sl6=df['sar_signal']
#     sl7=df['macd_signal']
#     sl8=df['stoch_signal']
#     sl9=df['obv_renko_signal']
#     sl10=df['aroon_signal']

#     for i in range(0,len(df)):
#         decision_d= ((w[0]*sl1[i]+w[1]*sl2[i]+w[2]*sl3[i]+w[3]*sl4[i]+w[4]*sl5[i]+w[5]*sl6[i]+w[6]*sl7[i]+w[7]*sl8[i]+w[8]*sl9[i]+w[9]*sl10[i])/sum(w))
#         if i!=0:
#             if df['adj_close_price'][i]<df['adj_close_price'][i-1]*0.8:
#                     mon+=amount*df['Close'][i]*0.998
#                     amount=0
#                     sell_idx.append(i)
#         if decision_d>td:
#             sumall=mon//df['adj_close_price'][i]
#             amount+=sumall
#             mon-=sumall*df['adj_close_price'][i]*1.002
#             count1+=1
#             buy_idx.append(i)
#         elif decision_d<-td:
#             mon+=amount*df['adj_close_price'][i]*0.998
#             amount=0
#             count2+=1
#             sell_idx.append(i)
#         else:
#             count3+=1
#     decision_d_l.append(mon+amount*df['adj_close_price'][i])
#     il.append(i)
#     ret=mon+amount*df['adj_close_price'][len(df)-1]
#     print (("TEST",b,ret-100000)/1000)


In [ ]:
fobj()

In [27]:
# coll=[[[0.08492177, 0.17384463, 0.01297783, 0.02911209, 0.00305128,
#        0.29679029, 0.09341777, 0.03109278, 0.2669813 , 0.00781026,
#        0.15002345], 77.78606595899961, 0],[[0.10124751, 0.07947271, 0.14059461, 0.01134871, 0.07670483,
#        0.0105102 , 0.01523628, 0.0793311 , 0.32688289, 0.15867116,
#        0.16469997], 29.423729452217085, 1],[[0.10424396, 0.04083642, 0.14153184, 0.04832486, 0.02425143,
#        0.23456657, 0.01785653, 0.04702162, 0.1687655 , 0.17260126,
#        0.23068131], 20.744831571363452, 2],[[0.03418246, 0.03985494, 0.05818124, 0.01006821, 0.05736128,
#        0.51578311, 0.04809931, 0.05745843, 0.05766466, 0.12134635,
#        0.13210764], 17.658730539011014, 3],[[0.04441311, 0.00105711, 0.10547514, 0.24048136, 0.00218894,
#        0.03192681, 0.0093497 , 0.09019205, 0.3072222 , 0.16769358,
#        0.2397252 ], 38.12829733082201, 4],[[0.15608604, 0.46143613, 0.02077728, 0.15747731, 0.00652082,
#        0.03187775, 0.00144456, 0.02996531, 0.12415056, 0.01026426,
#        0.05789345], 34.24385590455824, 5],[[0.03761342, 0.34845239, 0.11776582, 0.00239228, 0.03168897,
#        0.02931779, 0.15102881, 0.08606771, 0.11747511, 0.07819771,
#        0.18479926], 27.365418176465987, 6],[[0.03660642, 0.05033639, 0.12932883, 0.07908549, 0.01884995,
#        0.00878074, 0.15192304, 0.25559357, 0.00167098, 0.2678246 ,
#        0.22253394], 48.53843252603145, 7],[[0.00772096, 0.47030788, 0.07248883, 0.04361143, 0.25529928,
#        0.04781215, 0.00984386, 0.00458237, 0.02302817, 0.06530506,
#        0.11147168], 92.85205689248085, 8],[[0.02404885, 0.11613386, 0.04691288, 0.15394344, 0.01931855,
#        0.0351663 , 0.15083233, 0.02116866, 0.27700484, 0.15547028,
#        0.28969039], 78.26595460097695, 9]]

In [146]:
#test
import pandas as pd
sums=0
listna=[]
for j in range(0,11): #เเต่ละ signal 
    for i in range(0,10): #เเต่ละ data
        sums+=coll[i][0][j]
    sums=sums/10
    listna.append(sums)
tdna=listna[10]
w=listna[0:10]
dfo=pd.read_csv('TSLA_1417_year_testing_signal.csv')
count1,count2,count3=0,0,0
decision_d_l=[]
il=[]
sell_idx,buy_idx=[],[]
def testing(td,w,df):
    global count1,count2,count3,sell_idx,buy_idx,sl1
    mon=100000
    amount=0
    sumall=0
    sl1=dfo['sma89_signal']
    sl2=dfo['obv_macd_signal']
    sl3=dfo['cdc_signal']
    sl4=dfo['rsi_signal']
    sl5=dfo['renko_signal']
    sl6=dfo['sar_signal']
    sl7=dfo['macd_signal']
    sl8=dfo['stoch_signal']
    sl9=dfo['obv_renko_signal']
    sl10=dfo['aroon_signal']
    
    for i in range(0,len(df)):
        decision_d= ((w[0]*sl1[i]+w[1]*sl2[i]+w[2]*sl3[i]+w[3]*sl4[i]+w[4]*sl5[i]+w[5]*sl6[i]+w[6]*sl7[i]+w[7]*sl8[i]+w[8]*sl9[i]+w[9]*sl10[i])/sum(w))
        if i!=0:
            if df['adj_close_price'][i]<df['adj_close_price'][i-1]*0.8:
                    mon+=amount*dfo['Close'][i]*0.998
                    amount=0
                    sell_idx.append(i)
        if decision_d>td:
            sumall=mon//df['adj_close_price'][i]
            amount+=sumall
            mon-=sumall*df['adj_close_price'][i]*1.002
            count1+=1
            buy_idx.append(i)
        elif decision_d<-td:
            mon+=amount*df['adj_close_price'][i]*0.998
            amount=0
            count2+=1
            sell_idx.append(i)
        else:
            count3+=1
        decision_d_l.append(mon+amount*df['adj_close_price'][i])
        il.append(i)
    ret=mon+amount*df['adj_close_price'][len(df)-1]
    return (ret-100000)/1000
testing(tdna,w,dfo)

8.7122626862755

In [147]:
(dfo['adj_close_price'][len(dfo)-1]-dfo['adj_close_price'][0])/dfo['adj_close_price'][0]*100

107.4283664979756

In [150]:
#test
import pandas as pd
sums=0
listna=[]
for j in range(0,11): #เเต่ละ signal 
    for i in range(0,10): #เเต่ละ data
        sums+=coll[i][0][j]
    sums=sums/10
    listna.append(sums)
tdna=listna[10]
w=listna[0:10]
dfo=pd.read_csv('TSLA_18_year_testing_signal.csv')
count1,count2,count3=0,0,0
decision_d_l=[]
il=[]
sell_idx,buy_idx=[],[]
def testing(td,w,df):
    global count1,count2,count3,sell_idx,buy_idx,sl1
    mon=100000
    amount=0
    sumall=0
    sl1=dfo['sma89_signal']
    sl2=dfo['obv_macd_signal']
    sl3=dfo['cdc_signal']
    sl4=dfo['rsi_signal']
    sl5=dfo['renko_signal']
    sl6=dfo['sar_signal']
    sl7=dfo['macd_signal']
    sl8=dfo['stoch_signal']
    sl9=dfo['obv_renko_signal']
    sl10=dfo['aroon_signal']
    
    for i in range(0,len(df)):
        decision_d= ((w[0]*sl1[i]+w[1]*sl2[i]+w[2]*sl3[i]+w[3]*sl4[i]+w[4]*sl5[i]+w[5]*sl6[i]+w[6]*sl7[i]+w[7]*sl8[i]+w[8]*sl9[i]+w[9]*sl10[i])/sum(w))
        if i!=0:
            if df['adj_close_price'][i]<df['adj_close_price'][i-1]*0.8:
                    mon+=amount*dfo['Close'][i]*0.998
                    amount=0
                    sell_idx.append(i)
        if decision_d>td:
            sumall=mon//df['adj_close_price'][i]
            amount+=sumall
            mon-=sumall*df['adj_close_price'][i]*1.002
            count1+=1
            buy_idx.append(i)
        elif decision_d<-td:
            mon+=amount*df['adj_close_price'][i]*0.998
            amount=0
            count2+=1
            sell_idx.append(i)
        else:
            count3+=1
        decision_d_l.append(mon+amount*df['adj_close_price'][i])
        il.append(i)
    ret=mon+amount*df['adj_close_price'][len(df)-1]
    return (ret-100000)/1000
testing(tdna,w,dfo)

-3.8563480945091433

In [149]:
(dfo['adj_close_price'][len(dfo)-1]-dfo['adj_close_price'][0])/dfo['adj_close_price'][0]*100

4.161849649068813

In [151]:
listna

[0.05171901078056007,
 0.09168125927799935,
 0.0635959532800144,
 0.19770249244774873,
 0.06882487224742326,
 0.12934489843397567,
 0.07462249921901046,
 0.14327630557640067,
 0.14077705830467716,
 0.1346100853889713,
 0.1493072239607629]

In [63]:
listna

[0.03418788291218656,
 0.03149076244234548,
 0.15037019569028254,
 0.1325433769862197,
 0.14793535036282862,
 0.08853880692902165,
 0.09635943910162295,
 0.12403901528853958,
 0.12794771591086848,
 0.15992870893847594,
 0.19253667530824434]

In [ ]:
#BH 16 107.4283664979756
#back 16 110.85032925866324
#BH 4.161849649068813
#Forward 0

In [16]:
listna=[0.07209369806093725,
 0.07185038480865677,
 0.07605418723127022,
 0.046688922200022735,
 0.05800967858322942,
 0.20641780081608485,
 0.13221615163816813,
 0.17658586648082206,
 0.13257461186240724,
 0.12475782848656121,
 0.23315362570785392]

In [170]:
a=[0.07209369806093725,
 0.07185038480865677,
 0.07605418723127022,
 0.046688922200022735,
 0.05800967858322942,
 0.20641780081608485,
 0.13221615163816813,
 0.17658586648082206,
 0.13257461186240724,
 0.12475782848656121,
 0.23315362570785392]
td=a[10]
w=a[0:10]
df=splittest[0]

decision_d_l=[]
il=[]
global count1,count2,count3,sell_idx,buy_idx,sl1
mon=100000
amount=0
sumall=0
df=df.reset_index()
sl1=df['sma89_signal']
sl2=df['obv_macd_signal']
sl3=df['cdc_signal']
sl4=df['rsi_signal']
sl5=df['renko_signal']
sl6=df['sar_signal']
sl7=df['macd_signal']
sl8=df['stoch_signal']
sl9=df['obv_renko_signal']
sl10=df['aroon_signal']

for i in range(0,len(df)):
    decision_d= ((w[0]*sl1[i]+w[1]*sl2[i]+w[2]*sl3[i]+w[3]*sl4[i]+w[4]*sl5[i]+w[5]*sl6[i]+w[6]*sl7[i]+w[7]*sl8[i]+w[8]*sl9[i]+w[9]*sl10[i])/sum(w))
    if i!=0:
        if df['adj_close_price'][i]<df['adj_close_price'][i-1]*0.8:
                mon+=amount*df['Close'][i]*0.998
                amount=0
                sell_idx.append(i)
    if decision_d>td:
        sumall=mon//df['adj_close_price'][i]
        amount+=sumall
        mon-=sumall*df['adj_close_price'][i]*1.002
        count1+=1
        buy_idx.append(i)
    elif decision_d<-td:
        mon+=amount*df['adj_close_price'][i]*0.998
        amount=0
        count2+=1
        sell_idx.append(i)
    else:
        count3+=1
decision_d_l.append(mon+amount*df['adj_close_price'][i])
il.append(i)
ret=mon+amount*df['adj_close_price'][len(df)-1]
print (("TEST",b,ret-100000)/1000)

NameError: name 'b' is not defined

In [168]:
sl1

155    1
156    1
157    1
158   -1
159   -1
160    1
161    1
162    1
163    1
164    1
165   -1
166   -1
167   -1
168   -1
169   -1
170   -1
171   -1
172   -1
173   -1
174   -1
175   -1
176   -1
177   -1
178   -1
179   -1
180   -1
181   -1
182   -1
183   -1
184   -1
185   -1
Name: sma89_signal, dtype: int64